In [23]:
from functools import reduce
import os
from operator import or_, and_, contains

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from dateutil.parser import parse
import django
from django.db.models import Q
from flask_caching import Cache
#from jupyter_plotly_dash import JupyterDash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from toolz import keyfilter, merge, isiterable, get_in

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
# from plotter.views import *
# from plotter.forms import *
from utils import rows, columns, eta_methods, keygrab, in_me

In [2]:
app = dash.Dash()
CACHE_CONFIG = {
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': './.cache'
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

# we're using this to create thread-safe global values.
# also caching relatively expensive things like database lookups to the filesystem.
# is this actually slower? maybe. if so we will supplement with storing
# data in divs.
# we could instead just memoize to memory using lru_cache or similar

def cache_set(cache):
    def cset(key, value):
        return cache.set(key, value)
    return cset

def cache_get(key):
    def cget(key):
        return cache.get(key)
    return cget

In [3]:
def main_graph():
    return dcc.Graph(
            id = 'main-graph',
            figure = go.Figure(),
            style={'height':'100vh'}
        )

In [5]:
axis_value_properties = [
    {"label":"band average","value":"band_avg","type":"method","arity":2},
    {"label":"band maximum","value":"band_max","type":"method","arity":2},
    {"label":"band minimum","value":"band_min","type":"method","arity":2},
    {"label":"ratio","value":"ref_ratio","type":"method","arity":2},
    {"label":"band depth at middle filter","value":"band_depth_custom","type":"method","arity":3},
    {"label":"band depth at band minimum", "value":"band_depth_min","type":"method","arity":2},
    {"label":"band value", "value":"ref","type":"method","arity":1},
    {"label":"sol", "value":"sol","type":"parent_property"}
    ]

In [6]:
def make_axis(axis_value_properties, settings, queryset, suffix):
    
    # grab text for everything in queryset
    
    # what is requested function or property?
    axis_option = settings["axis-option-"+suffix]
    # what are the characteristics of that function or property?
    props = keygrab(axis_value_properties, "value", axis_option)
    if props["type"] == "method":
        # we assume here that 'methods' all take a spectrum's filter names
        # as arguments, and have arguments in an order corresponding to the inputs.
        filt_args = [
            settings['filter-'+str(ix)+'-'+suffix] 
            for ix in range(1,props["arity"]+1)
        ]
        # if some values are blank, don't try to call the function
        if all(filt_args):
            return [
                getattr(spectrum,props["value"])(*filt_args) 
                for spectrum in queryset
            ]
        return None
    # do some other stuff to grab parent properties

In [7]:
def scatter(x_axis, y_axis):
    fig = go.Figure()
    fig.add_trace(go.Scattergl(
        x = x_axis,
        y = y_axis,
        # change this to be hella popup text
        text = None,
        mode='markers',
        marker = {
            'color':'blue'
        },
        ))
    return fig

In [8]:
def flexible_query(queryset, field, value):
    """
    little search function that checks exact and loose phrases
    """
    # allow exact phrase searches
    query = field + "__iexact"
    if queryset.filter(**{query: value}):
        return queryset.filter(**{query: value})
    # otherwise treat multiple words as an 'or' search
    query = field + "__icontains"
    filters = [
        queryset.filter(**{query: word}) for word in value.split(" ")
    ]
    return reduce(or_, filters)

In [9]:
def particular_fields_search(model, search_dict, searchable_fields):
    """
    'search specific defined fields' function.
    works only on strings atm!
    other things for numeric / date / etc. fields preferably
    
    'searchable fields' is a little clumsy -- but it specifically
    makes the function a little more fault-tolerant.
    """
    queryset = model.objects.all()
    # allow either single entries or lists of entries
    for field in searchable_fields:
        entry = search_dict.get(field)
        if entry:
            entry = list(entry)
            filters = [flexible_query(queryset, field, value) for value in entry]
            queryset = reduce(or_,filters)
    return queryset

In [10]:
def filter_drop(model, element_id):
    """dropdown for filter selection"""
    return dcc.Dropdown(
        id=element_id,
        options=[{'label': filt, 'value': filt}
                 for filt in model.filters],
        style = {'width':'10rem', 'display':'inline-block'}
    )

In [11]:
def field_drop(fields, element_id):
    """dropdown for field selection -- no special logic atm"""
    return dcc.Dropdown(
        id=element_id,
        options = [{'label': field, 'value': field}
                 for field in fields]
    )

In [12]:
def field_values(queryset, field):
    """
    generates dict if all unique values in model's field
    + any and blank, for passing to HTML select constructors
    as this is based on current queryset,
    it will by default display options as constrained by other search
    parameters. this has upsides and downsides.
    it will also lead to odd behavior if care is not given.
    maybe it's a bad idea.
    """
    options_list = [
            {'label': item, 'value': item}
            for item in set(qlist(queryset,field))
            if item not in ['','nan']
        ]
    special_options = [
        {'label':'any',  'value':'any'},
        # too annoying to detect all 'blank' things atm
        # {'label':'no assigned value','value':''}
    ]
    return special_options + options_list


def model_options_drop(queryset, field, element_id):
    """
    dropdown for selecting search values for a specific field
    could end up getting unmanageable as a UI element
    """
    return dcc.Dropdown(
        id=element_id,
        options=field_values(queryset, field),
        multi = True
    )

In [13]:
def handle_search(model, search_dict, searchable_fields):
    """
    dispatcher. right now just handles 'no assigned'
    and 'any' cases
    """
    for field, value in search_dict.items():
        if 'any' in value:
            return model.objects.all()
    return particular_fields_search(model, search_dict, searchable_fields)

('1', '8')

In [14]:
def axis_value_drop(axis_value_properties,element_id):
    """
    dropdown for selecting calculation options for axes
    """
    options=[
            {"label":option["label"], "value":option["value"]}
             for option in axis_value_properties
        ]
    return dcc.Dropdown(
        id=element_id,
        options=options,
        value = options[0]["value"]
    )

In [15]:
for function in [
    handle_search, 
    particular_fields_search,
    flexible_query,
    qset_axes,
    qlist,
    make_axis
    ]:
    function = cache.memoize()(function)

In [16]:
fig = main_graph()
graph_function = scatter
spec_model = MSpec 
obs_model = MObs
# update this -- a buncha fields, both spec and obs
searchable_fields = ['group','formation','member']
# active queryset is explicitly stored in global cache
cset('queryset', spec_model.objects.all().prefetch_related("observation"))

# it's a big open UX question how to gracefully mutate filters between instruments.
# (i.e., if we want to be able to display spectra from multiple instruments
# simultaneously rather than successively).
# it might be that flexible band values are actually better than explicit filter
# selection. for now we're using this, though.
app.layout = html.Div(children = [
    html.Div(children = [
        axis_value_drop('axis-option-x'),
        filter_drop(spec_model,'filter-1-x'),
        filter_drop(spec_model,'filter-3-x'),
        filter_drop(spec_model,'filter-2-x'),
    ]),
    html.Div(children = [
        axis_value_drop('axis-option-y'),
        filter_drop(spec_model,'filter-1-y'),
        filter_drop(spec_model,'filter-3-y'),
        filter_drop(spec_model,'filter-2-y'),
    ]),
    html.Div(id = 'calculation-state', style={"display":"none"}),
    html.Div(id = 'search-trigger', style={"display":"none"}),
    # will probably want to split divs off here so we can generate new search rows
    # how do we manage that in callbacks? is there a name selector?
    # yes -- see https://dash.plotly.com/pattern-matching-callbacks
    field_drop(searchable_fields, 'field-search'),
    model_options_drop(cget('queryset'), 'group', 'value-search'),
    html.Button(id='submit-search', n_clicks=0, children='Submit'),
    html.Div(children = [fig]),
    ])

In [17]:
main_graph = Output('main-graph','figure')

x_inputs = [
    Input('filter-1-x', 'value'),
    Input('filter-2-x', 'value'),
    Input('filter-3-x', 'value'),
    Input('axis-option-x', 'value'),
]
y_inputs = [
    Input('filter-1-y', 'value'),
    Input('filter-2-y', 'value'),
    Input('filter-3-y', 'value'),
    Input('axis-option-y', 'value')
]

In [18]:
def pickitems(dictionary, some_list):
    """items of dict where key is in some_list """
    return keyfilter(in_me(some_list),(dictionary))

def pickcomps(comp_dictionary, id_list):
    """items of dictionary of dash components where id is in id_list"""
    return pickitems(comp_dictionary, [comp.component_id for comp in comp_dictionary])

def pickctx(context, component_list):
    """states and inputs of dash callback context if component is in component_list"""
    comp_strings = [
        comp.component_id+'.'+comp.component_property 
        for comp in component_list
    ]
    cats = [] 
    if context.states:
        cats.append(context.states)
    if context.inputs:
        cats.append(context.inputs)
    picked = [pickitems(cat, comp_strings) for cat in cats]
    if picked:
        return merge(picked)

In [19]:
def change_input_visibility(axis_value_properties, calc_type):
    """
    turn visibility of filter dropdowns (and later other inputs)
    on and off in response to changes in arity / type of 
    requested calc
    """
    props = keygrab(axis_value_properties, "value", calc_type)
    # 'methods' are specifically those methods of spec_model
    # that take its filters as arguments
    if props["type"] == "method":
        return [
            {'width':'10rem', 'display':'inline-block'} if x < props["arity"]
            else {'width':'10rem', 'display':'none'}
            for x in range(3)
        ]
    return [
        {"display":"none"} for x in range(3)
    ]


# register vis control functions with appropriate i/o
for axis in ['x', 'y']:
    app.callback(
        [
            Output('filter-1-'+axis, 'style'),
            Output('filter-2-'+axis, 'style'),
            Output('filter-3-'+axis, 'style'),
        ],
        [Input('axis-option-'+axis, 'value')]
    )(change_input_visibility)
    
    
# this is somewhat nasty.
# is there a cleaner way to do this?
# flow control becomes really hard if we break the function up.
# it requires triggers spread across multiple divs or cached globals
# and is much uglier than even this
@app.callback(
    Output('main-graph', 'figure'),
    # maybe later add an explicit recalc button?
    [*x_inputs, *y_inputs, Input('search-trigger', 'value')],
)
def recalculate_graph(*args,x_inputs,y_inputs):
    ctx = dash.callback_context
    # do nothing on page load
    if not ctx.triggered:
        raise PreventUpdate
    queryset = cget("queryset")
    x_settings = pickctx(ctx, x_inputs)
    y_settings = pickctx(ctx, y_inputs)
    # this is for future functions that display or recall 
    # the settings used to generate a graph
    cache.set("x_settings",x_settings)
    cache.set("y_settings",y_settings)
    # we can probably get away without any fancy flow control
    # b/c memoization...if it turns out that hitting the cache this way sucks,
    # we will add it.
    x_axis = make_axis(x_settings, queryset, suffix='x.value')
    y_axis = make_axis(y_settings, queryset, suffix='y.value')
    # this case is most likely shortly after page load
    # when not everything is filled out
    if not (x_axis and y_axis):
        raise PreventUpdate
    return graph_function(x_axis, y_axis)   


@app.callback(Output('value-search', 'options'),
              [Input('field-search', 'value')],
             )
def update_model_field(field):
    """populate set of values when field in search box changes"""
    queryset = cget("queryset")
    return field_values(queryset, field)

    
@app.callback(Output('search-trigger', 'value'),
              [Input('submit-search', 'n_clicks')],
              [State('field-search', 'value'),
               State('value-search', 'value'),
              ])
def update_queryset(n_clicks, field, value):
    """
    updates the spectra displayed in the graph view.
    
    we'd actually like to extend this to include fields from both spec
    and obs
    
    and multiple fields
    """
    # don't do anything on page load
    # or if a partially blank request is issued
    if not (field and value):
        raise PreventUpdate
    
    # if the search parameters have changed,
    # make a new queryset and trigger graph update
    if handle_search(
        spec_model, {field:value}, [field]
    ) != cget("queryset"):
        cset(
            "queryset", handle_search(
                spec_model, {field:value}, [field]
            ).prefetch_related("observation")
        )
        return n_clicks

In [20]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jul/2020 00:03:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 00:03:51] "POST /_dash-update-component HTTP/1.1" 204 -


j


127.0.0.1 - - [01/Jul/2020 00:03:52] "POST /_dash-update-component HTTP/1.1" 204 -


j


...

j


127.0.0.1 - - [01/Jul/2020 00:03:57] "POST /_dash-update-component HTTP/1.1" 200 -


j


127.0.0.1 - - [01/Jul/2020 00:03:58] "POST /_dash-update-component HTTP/1.1" 200 -


j


In [21]:
app.callback_map

{'..filter-1-x.style...filter-2-x.style...filter-3-x.style..': {'inputs': [{'id': 'axis-option-x',
    'property': 'value'}],
  'state': [],
  'callback': <function __main__.change_input_visibility(calc_type)>},
 '..filter-1-y.style...filter-2-y.style...filter-3-y.style..': {'inputs': [{'id': 'axis-option-y',
    'property': 'value'}],
  'state': [],
  'callback': <function __main__.change_input_visibility(calc_type)>},
 'main-graph.figure': {'inputs': [{'id': 'filter-1-x', 'property': 'value'},
   {'id': 'filter-2-x', 'property': 'value'},
   {'id': 'filter-3-x', 'property': 'value'},
   {'id': 'axis-option-x', 'property': 'value'},
   {'id': 'filter-1-y', 'property': 'value'},
   {'id': 'filter-2-y', 'property': 'value'},
   {'id': 'filter-3-y', 'property': 'value'},
   {'id': 'axis-option-y', 'property': 'value'},
   {'id': 'search-trigger', 'property': 'value'}],
  'state': [],
  'callback': <function __main__.recalculate_graph(*args)>},
 'value-search.options': {'inputs': [{'id': 